<a href="https://www.kaggle.com/code/dewanrapa/senti-ana-fulldata-w-bert?scriptVersionId=202890002" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!nvidia-smi

Wed Oct 23 16:44:38 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import time
from joblib import Parallel, delayed
import multiprocessing

In [4]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Set the model to use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Check if using GPU
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

Using GPU: Tesla P100-PCIE-16GB


In [5]:

url = 'https://raw.githubusercontent.com/amanullahshah32/Review-Scraping/refs/heads/main/Dataset/all_app_reviews.csv'
df = pd.read_csv(url)

In [6]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)

In [7]:
df_review_description = df_shuffled[['review_description']]


In [8]:
full_df = df[['review_description']]

In [9]:
full_df

,review_description
0,nice
1,good
2,good
3,good
4,good 😊
...,...
99390,wow
99391,র্গববতী গরু খিচনি রোগ চিকিছ্যা পরামর্শ
99392,good
99393,"ফালতু অ্যাপ, ওষুধের ডোজ কতটুকু তা দেওয়া নাই"


In [10]:
df_review_description.head()

,review_description
0,Excellent app for learning
1,Is helping me to quit this ugly habit
2,Excelent
3,Noice
4,Ripon


In [11]:

def sentiment_score(review):
    # Handle missing or non-string values by converting to empty strings
    if not isinstance(review, str):
        review = ''
    
    # Tokenize and move the tokens to GPU
    tokens = tokenizer.encode(review, return_tensors='pt').to(device)
    
    # Perform inference on GPU
    result = model(tokens)
    
    # Return the sentiment score (1-5 scale)
    return int(torch.argmax(result.logits)) + 1

In [15]:
# Step 1: Clean the DataFrame by replacing non-string values
full_df['review_description'] = full_df['review_description'].apply(lambda x: x if isinstance(x, str) else '')

# Step 2: Measure the start time
start_time = time.time()

# Step 3: Apply sentiment scoring
full_df['sentiment_full_df'] = df_review_description['review_description'].apply(lambda review: sentiment_score(review[:512]))

# Step 4: Measure the end time and print the total time taken
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

/tmp/ipykernel_30/3721606111.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df['review_description'] = full_df['review_description'].apply(lambda x: x if isinstance(x, str) else '')


Time taken: 933.9244675636292 seconds


/tmp/ipykernel_30/3721606111.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df['sentiment_full_df'] = df_review_description['review_description'].apply(lambda review: sentiment_score(review[:512]))


In [16]:
full_df.sentiment_full_df.value_counts()

sentiment_full_df
5    57141
4    25077
1     8174
3     6533
2     2470
Name: count, dtype: int64

In [17]:
full_df.sentiment_full_df.isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
99390    False
99391    False
99392    False
99393    False
99394    False
Name: sentiment_full_df, Length: 99395, dtype: bool

In [18]:
# Step 5: Remove rows where sentiment_full_df is null
full_df_cleaned = full_df.dropna(subset=['sentiment_full_df'])

# Verify that no null values are present
print(f"Number of nulls in 'sentiment_full_df': {full_df_cleaned['sentiment_full_df'].isnull().sum()}")

# Optional: Show the shape of the cleaned DataFrame to see how many rows are left
print(f"Cleaned DataFrame shape: {full_df_cleaned.shape}")

Number of nulls in 'sentiment_full_df': 0
Cleaned DataFrame shape: (99395, 2)


In [19]:
full_df_cleaned

,review_description,sentiment_full_df
0,nice,5
1,good,5
2,good,5
3,good,2
4,good 😊,1
...,...,...
99390,wow,1
99391,র্গববতী গরু খিচনি রোগ চিকিছ্যা পরামর্শ,3
99392,good,5
99393,"ফালতু অ্যাপ, ওষুধের ডোজ কতটুকু তা দেওয়া নাই",5


In [25]:
 full_df_cleaned['sentiment_full_df'].value_counts()

sentiment_full_df
5    57141
4    25077
1     8174
3     6533
2     2470
Name: count, dtype: int64